### [AsyncIO for the Working Python Developer](https://hackernoon.com/asyncio-for-the-working-python-developer-5c468e6e2e8e#.q6900g3w2)
                                           
Then I found [asyncio](https://docs.python.org/dev/library/asyncio.html), and everything changed.

- **event loop**: manages and distributes the execution of different tasks

- **Coroutines**: generators that on yield they release the flow of control back to the event loop. A coroutine needs to be scheduled to run using the event loop, to do this we create a *Task*, which is a type of Future.

- **Futures**: objects that represent the result of a task that may or may not have been executed. 

[Concurrency is not parallelism, it’s better](https://www.youtube.com/watch?v=cN_DpYBzKso)
- **Parallelism**: just breaking down tasks into concurrent subtasks.
- **Concurrency**: the scheduling of these subtasks that creates parallelism.

**Asyncio** dose that concurrency, with structuring subtasks are defined as coroutines and allows you to schedule them, including simutaneously.

### Synchronous & Asynchronous Execution
**yield** from on another coroutine:
 - give the control back to the event loop
 - in this case `sleep` will yield and 
 - the event loop will switch contexts to the next task scheduled for execution: bar 

In [1]:
import asyncio

@asyncio.coroutine
def foo():
    print('Runing in foo')
    yield from asyncio.sleep(0)
    print ('Explicit context switch to foo again')
    
@asyncio.coroutine
def bar():
    print('Explicit context to bar')
    yield from asyncio.sleep(0)
    print('Implicit context switch back to bar')
    
ioloop = asyncio.get_event_loop()
tasks = [ioloop.create_task(foo()), ioloop.create_task(bar())]
wait_tasks = asyncio.wait(tasks)
ioloop.run_until_complete(wait_tasks)
#ioloop.close()

Runing in foo
Explicit context to bar
Explicit context switch to foo again
Implicit context switch back to bar


({<Task finished coro=<foo() done, defined at <ipython-input-1-035a604a3e03>:3> result=None>,
  <Task finished coro=<bar() done, defined at <ipython-input-1-035a604a3e03>:9> result=None>},
 set())

> While the two blocking tasks are blocked a third one can take control of the flow.

In [2]:
import time
import asyncio

start = time.time()

def tic():
    return 'at %1.1f seconds' % (time.time() - start)

@asyncio.coroutine
def gr1():
    # Busy waits for a second, but we don't want to stick around...
    print('gr1 started work: {}'.format(tic()))
    yield from asyncio.sleep(2)
    print('gr1 ended work: {}'.format(tic()))

@asyncio.coroutine
def gr2():
    # Busy waits for a second, but we don't want to stick around...
    print('gr2 started work: {}'.format(tic()))
    yield from asyncio.sleep(2)
    print('gr2 Ended work: {}'.format(tic()))

@asyncio.coroutine
def gr3():
    print("Let's do some stuff while the coroutines are blocked, {}".format(tic()))
    yield from asyncio.sleep(1)
    print("Done!")
    
ioloop = asyncio.get_event_loop()
tasks = [
    ioloop.create_task(gr1()),
    ioloop.create_task(gr2()),
    ioloop.create_task(gr3())
]
ioloop.run_until_complete(asyncio.wait(tasks))
#ioloop.close()

gr1 started work: at 0.0 seconds
gr2 started work: at 0.0 seconds
Let's do some stuff while the coroutines are blocked, at 0.0 seconds
Done!
gr1 ended work: at 2.0 seconds
gr2 Ended work: at 2.0 seconds


({<Task finished coro=<gr3() done, defined at <ipython-input-2-023dec6d6210>:23> result=None>,
  <Task finished coro=<gr2() done, defined at <ipython-input-2-023dec6d6210>:16> result=None>,
  <Task finished coro=<gr1() done, defined at <ipython-input-2-023dec6d6210>:9> result=None>},
 set())

### Order of execution
Some tasks finish in different order than they were scheduled. asyncio does not magically make things non-blocking.

In [3]:
import random
from time import sleep
import asyncio

def task(pid):
    """
    Some non-deterministic task
    """
    sleep(random.randint(0, 2) * 0.001)
    print('Task %s done' % pid)
    
@asyncio.coroutine
def task_coro(pid):
    """
    Some non-deterministic task
    """
    yield from asyncio.sleep(random.randint(0, 2) * 0.001)
    print('Task %s done' % pid)

    
def synchronous():
    for i in range(1, 10):
        task(i)

        
@asyncio.coroutine
def asynchronous():
    tasks = [asyncio.async(task_coro(i)) for i in range(1, 10)]
    yield from asyncio.wait(tasks)


print('Synchronous:')
synchronous()

ioloop = asyncio.get_event_loop()
print('Asynchronous:')
ioloop.run_until_complete(asynchronous())

Synchronous:
Task 1 done
Task 2 done
Task 3 done
Task 4 done
Task 5 done
Task 6 done
Task 7 done
Task 8 done
Task 9 done
Asynchronous:
Task 4 done
Task 5 done
Task 3 done
Task 6 done
Task 7 done
Task 8 done
Task 1 done
Task 2 done
Task 9 done


> The result is that requesting and retrieving the result of all requests takes only as long as the slowest request.

In [4]:
import time
import urllib.request
import asyncio
import aiohttp

URL = 'https://api.github.com/events'
MAX_CLIENTS = 3

def fetch_sync(pid):
    print('Fetch sync process {} started'.format(pid))
    start = time.time()
    response = urllib.request.urlopen(URL)
    datetime = response.getheader('Date')

    print('Process {}: {}, took: {:.2f} seconds'.format(pid, datetime, time.time() - start))
    
    return datetime

@asyncio.coroutine
def fetch_async(pid):
    print('Fetch async process {} started.'.format(pid))
    start = time.time()
    response = yield from aiohttp.request('GET', URL)
    datetime = response.headers.get('Date')
    
    print('Process {}: {}, took: {:.2f} seconds'.format(pid, datetime, time.time() - start))
    
    response.close()
    return datetime

def synchronous():
    start = time.time()
    for i in range(1, MAX_CLIENTS + 1):
        fetch_sync(i)
    print("Process took: {:.2f} seconds".format(time.time() - start))
    
@asyncio.coroutine
def asynchronous():
    start = time.time()
    # gathering the corouintes into a list. 
    tasks = [asyncio.ensure_future(fetch_async(i)) for i in range(1, MAX_CLIENTS + 1)]
    yield from asyncio.wait(tasks)
    print("Process took: {:.2f} seconds".format(time.time() - start))

print('Synchronous:')
synchronous()

print('Asynchronous:')
ioloop = asyncio.get_event_loop()
ioloop.run_until_complete(asynchronous())
#ioloop.close()

Synchronous:
Fetch sync process 1 started
Process 1: Fri, 17 Feb 2017 14:33:17 GMT, took: 1.45 seconds
Fetch sync process 2 started
Process 2: Fri, 17 Feb 2017 14:33:18 GMT, took: 1.20 seconds
Fetch sync process 3 started
Process 3: Fri, 17 Feb 2017 14:33:20 GMT, took: 1.30 seconds
Process took: 3.96 seconds
Asynchronous:
Fetch async process 1 started.
Fetch async process 2 started.
Fetch async process 3 started.
Process 3: Fri, 17 Feb 2017 14:33:21 GMT, took: 1.12 seconds
Process 2: Fri, 17 Feb 2017 14:33:21 GMT, took: 1.25 seconds
Process 1: Fri, 17 Feb 2017 14:33:21 GMT, took: 1.32 seconds
Process took: 1.32 seconds


### Creating concurrency
Coroutines can be scheduled to run or retrieve their results in different ways. Imagine a scenario where we need to process the results of the HTTP GET requests as soon as they arrive.

In [5]:
import time
import random
import asyncio
import aiohttp

URL = 'https://api.github.com/events'
MAX_CLIENTS = 3

@asyncio.coroutine
def fetch_async(pid):
    start = time.time()
    sleepy_time = random.randint(2, 5)
    print('Fetch async process {} started, sleeping for {} seconds'
          .format(pid, sleepy_time))
    
    yield from asyncio.sleep(sleepy_time)
    
    response = yield from aiohttp.request('GET', URL)
    datetime = response.headers.get('Date')
    
    response.close()
    return 'Process {}: {}, took: {:.2f} seconds'.format(pid, datetime, time.time() - start)
    
@asyncio.coroutine
def asynchronous():
    start = time.time()
    futures = [fetch_async(i) for i in range(1, MAX_CLIENTS + 1)]
    for i, future in enumerate(asyncio.as_completed(futures)):
        result = yield from future
        print('{} {}'.format(">>" * (i + 1), result))
        
    print("Process took: {:.2f} seconds".format(time.time() - start))
    

ioloop = asyncio.get_event_loop()
ioloop.run_until_complete(asynchronous())
#ioloop.close()

Fetch async process 3 started, sleeping for 3 seconds
Fetch async process 1 started, sleeping for 2 seconds
Fetch async process 2 started, sleeping for 3 seconds
>> Process 1: Fri, 17 Feb 2017 14:33:32 GMT, took: 3.24 seconds
>>>> Process 2: Fri, 17 Feb 2017 14:33:33 GMT, took: 4.78 seconds
>>>>>> Process 3: Fri, 17 Feb 2017 14:33:33 GMT, took: 4.86 seconds
Process took: 4.86 seconds


> `as_completed` and `wait` are both functions originally from `concurrent.futures`.

Sending concurrent requests to each service and pick the first one that responds using parameter **`return_when`** of **`wait`** method.

In [7]:
from collections import namedtuple
import time
import asyncio
from concurrent.futures import FIRST_COMPLETED
import aiohttp

Service = namedtuple('Service', ('name', 'url', 'ip_attr'))

SERVICES = (
    Service('ipify', 'https://api.ipify.org?format=json', 'ip'),
    Service('ip-api', 'http://ip-api.com/json', 'query')
)

@asyncio.coroutine
def fetch_ip(service):
    start = time.time()
    print('Fetching IP from {}'.format(service.name))
    
    response = yield from aiohttp.request('GET', service.url)
    json_response = yield from response.json()
    ip = json_response[service.ip_attr]
    
    response.close()
    return '{} finished with result: {}, took: {:.2f} seconds'.format(
        service.name, ip, time.time() - start)


@asyncio.coroutine
def asynchronous():
    futures = [fetch_ip(service) for service in SERVICES]
    done, pending = yield from asyncio.wait(
        futures, return_when=FIRST_COMPLETED)
    print(done.pop().result())

    
ioloop = asyncio.get_event_loop()
ioloop.run_until_complete(asynchronous())
#ioloop.close()

Fetching IP from ip-api
Fetching IP from ipify
ip-api finished with result: 119.64.234.10, took: 1.05 seconds
